In [6]:
import requests
from bs4 import BeautifulSoup
import fitz  # PyMuPDF for reading PDFs
import ollama

# LinkedIn job search URL for "Data Analyst" (Modify location as needed)
LINKEDIN_URL = "https://www.linkedin.com/jobs/search?keywords=Data%20Analyst"

def fetch_jobs():
    """Scrape LinkedIn for Data Analyst jobs."""
    headers = {
        "User-Agent": "Mozilla/5.0"
    }
    response = requests.get(LINKEDIN_URL, headers=headers)
    if response.status_code != 200:
        print("Failed to fetch job listings.")
        return []
    
    soup = BeautifulSoup(response.text, "html.parser")
    jobs = []
    
    # Extract job titles and links (Modify based on actual LinkedIn page structure)
    for job_card in soup.find_all("div", class_="base-search-card"):  # Adjust this selector as needed
        title = job_card.find("h3").text.strip() if job_card.find("h3") else "Unknown"
        company = job_card.find("h4").text.strip() if job_card.find("h4") else "Unknown"
        link = job_card.find("a")["href"] if job_card.find("a") else "#"
        
        jobs.append({"title": title, "company": company, "link": link})
    
    return jobs

def read_resume(file_path):
    """Extract text from resume PDF."""
    doc = fitz.open(file_path)
    text = ""
    for page in doc:
        text += page.get_text()
    return text.strip()

def analyze_jobs_with_resume(jobs, resume_text):
    """Use Ollama to analyze job descriptions and generate a scorecard."""
    scorecard = []
    
    for job in jobs:
        job_desc = f"Job Title: {job['title']}\nCompany: {job['company']}\nJob Link: {job['link']}"
        prompt = f"Based on the following resume, score the candidate's fit (1-100) for this job.\n\nResume:\n{resume_text}\n\n{job_desc}\n\nGive a numerical score and a brief justification."
        
        response = ollama.chat(model="llama3.2", messages=[{"role": "user", "content": prompt}])
        analysis = response["message"]["content"] if "message" in response else "No response"
        
        scorecard.append({"job": job, "analysis": analysis})
    
    return scorecard

def main():
    resume_path = "C:/Users/PUSHKAR/OneDrive/Desktop/Pushkar_cv.pdf"  # Update with your actual resume file path
    jobs = fetch_jobs()
    
    if not jobs:
        print("No jobs found.")
        return
    
    resume_text = read_resume(resume_path)
    scorecard = analyze_jobs_with_resume(jobs, resume_text)
    
    # Display scorecard
    for entry in scorecard:
        print(f"Job: {entry['job']['title']} at {entry['job']['company']}")
        print(f"Score Analysis: {entry['analysis']}")
        print("-" * 50)

if __name__ == "__main__":
    main()

Job: Data Analyst at Zest AI
Score Analysis: Based on the resume, I would score the candidate's fit for the job as 82 out of 100.

Justification:

1. Relevant experience: The candidate has relevant experience in data analysis, having worked on projects such as Power BI reports, SQL analysis, and machine learning. This experience aligns with the requirements of the Data Analyst role at Zest AI.
2. Technical skills: The candidate has proficiency in technical tools such as Python, Power BI, SQL, Tableau, and Advanced Excel, which are relevant to the job requirements.
3. Education: The candidate has a postgraduate degree in Data Science and AI from IIT Roorkee, which is a prestigious institution and demonstrates a strong educational background.
4. Soft skills: The candidate's profile highlights their motivation, eagerness to learn, and ability to translate raw data into actionable insights, which are essential soft skills for a Data Analyst role.

However, the score is not perfect because: